In [1]:
import librosa
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa.display
import pathlib
import warnings
warnings.filterwarnings('ignore')

In [2]:
audio_dataset_path='D:/Hackthon/IA/dataset/TrainAudioFiles/'
df = pd.read_csv('train.csv')
df.head(10)

,filename,emotion
0,18777.mp3,neutral
1,24041.mp3,neutral
2,1621.mp3,joy
3,28883.mp3,neutral
4,14918.mp3,neutral
5,25793.mp3,neutral
6,8382.mp3,joy
7,50666.wav,disgust
8,28471.mp3,surprise
9,1459.mp3,neutral


In [3]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [4]:
extracted_features=[]
for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["filename"]))
    final_class_labels=row["emotion"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

5816it [34:08,  2.84it/s]


In [5]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head(10)

,feature,class
0,"[-283.32904, 126.99362, -36.856903, 43.745472,...",neutral
1,"[-291.48126, 85.95444, -31.580286, 28.471468, ...",neutral
2,"[-362.79108, 123.998245, -15.96158, 73.43797, ...",joy
3,"[-236.65076, 117.45328, -47.395912, 15.700452,...",neutral
4,"[-395.16562, 87.81874, -8.651618, 25.296888, -...",neutral
5,"[-389.05655, 115.418304, -30.919865, 38.674637...",neutral
6,"[-287.7145, 84.3601, -39.53423, 34.14563, -34....",joy
7,"[-436.434, 129.58134, 13.694379, 41.57397, -3....",disgust
8,"[-382.01114, 108.56655, -23.953827, 8.383499, ...",surprise
9,"[-262.59802, 119.459076, -13.18597, 44.373013,...",neutral


In [6]:
import pickle

In [7]:
with open('audio_df_Ac1.pickle', 'wb') as f:
    pickle.dump(extracted_features_df, f)

In [8]:
with open('audio_df_Ac1.pickle', 'rb') as f:
    extracted_features_df= pickle.load(f)

In [9]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=labelencoder.fit_transform(y)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [12]:
from sklearn.neighbors import KNeighborsClassifier

model1 = KNeighborsClassifier(n_neighbors=60)
model1.fit(X_train,y_train)

#Predict Output
predicted_label= model1.predict(X_test) # 0:Overcast, 2:Mild
prediction_class = labelencoder.inverse_transform(predicted_label)
prediction_class

array(['joy', 'neutral', 'neutral', ..., 'neutral', 'neutral', 'disgust'],
      dtype='<U8')

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicted_label)

0.4845360824742268

In [14]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [15]:
y_pred = gnb.predict(X_test)

In [16]:
accuracy_score(y_test, y_pred )

0.46735395189003437